# Fingerprints DB ELT Script

In [ ]:
import requests
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
import json

pd.set_option('display.max_rows', 150)

# Input Addresses and Desired Data

In [ ]:
# add relevant addresses below
addresses = {}

addresses['FP Main'] = '0xbc49de68bcbd164574847a7ced47e7475179c76b'
addresses['FP OPEX'] = '0xc9044dd6162106edb5ece3af8a8e8b657b612e42'
addresses['FP Treasury'] = '0xB28B71C694F53C7CfEA8dffDC85733237e0C981e'
# addresses['HW Phase3'] = '0x45e77e6473676ab30ed64f3815fea7847d02d7b8'
# addresses['HW Compensation'] = '0x7af805dcd0c18ed276f99d83197fe7a9d37b3c54'
# addresses['HW Panels Competition'] = '0xf358e6b648579Bc43042B3cc7661dD5309EB06b3'
# addresses['HW Avid Lines'] = '0x75a5B40075bc35bcD09f9d1f7d9c39Efd82B401c'
# addresses['HW Early Operations'] = '0x0c41409b8e024b762f07b13da602409be37ac4d5'


# don't mess with txtypes!
txtypes = {}

txtypes['normal'] = 'txlist'
txtypes['erc20'] = 'tokentx'
txtypes['nfts'] = 'tokennfttx'
txtypes['internal'] = 'txlistinternal'
txtypes['balance'] = 'balance'


# Get Last Block from Database

In [ ]:
from sqlalchemy import create_engine
load_dotenv('.env')
DB_CREDS = os.environ.get('DB_CREDS')
engine = create_engine(DB_CREDS, echo=True)


# get last block by txtype for each address from database
sql = {}
last_blocks = {}
for txtype in txtypes:
    sql[txtype] = {}
    last_blocks[txtype] = {}
    if txtype != 'balance':
        for address in addresses.values():
            sql[txtype][address] = f"select max(\"blockNumber\")+1 as block_number from financials.{txtype} where \"from\" = lower('{address}') or \"to\" = lower('{address}');"
            last_blocks[txtype][address] = pd.read_sql_query(sql[txtype][address], con=engine).iloc[0]['block_number']

last_blocks

# Requests - Transactions

In [ ]:
ETHERSCAN_APIKEY = os.environ.get('ETHERSCAN_APIKEY')

dfs = {}
merge = []
# get data from etherscan api
for txtype in txtypes:
    for address in addresses.values():
        if txtype == 'balance': # if querying for balance, different payload
            payload = {
                'module': 'account',
                'action': txtypes[txtype],
                'address': address,
                'tag': 'latest',
                'apikey': ETHERSCAN_APIKEY
            }
            r = requests.get('https://api.etherscan.io/api', params=payload)
            response = json.loads(r.text)['result']
            merge.append(pd.DataFrame([{'address': address, 'balance': response}]))
        else:
            if last_blocks[txtype][address] == None: # if address is new, start block = 0
                payload = {
                    'module': 'account',
                    'action': txtypes[txtype],
                    'address': address,
                    'startblock': '0',
                    'endblock': '999999999',
                    'sort': 'asc',
                    'apikey': ETHERSCAN_APIKEY
                }
            else: # else, use last block from database
                payload = {
                    'module': 'account',
                    'action': txtypes[txtype],
                    'address': address,
                    'startblock': last_blocks[txtype][address],
                    'endblock': '999999999',
                    'sort': 'asc',
                    'apikey': ETHERSCAN_APIKEY
                }
            r = requests.get('https://api.etherscan.io/api', params=payload)
            response = json.loads(r.text)['result']
            merge.append(pd.DataFrame.from_dict(response))
            

    dfs[txtype] = pd.concat(merge) # store all merged data in a df
    dfs[txtype] = dfs[txtype].drop_duplicates() # remove duplicates (txs between our own wallets)
    merge = [] # reset merge array to store new txtype data


# Save to CSVs

In [ ]:
# ----------------------------------------------------------------------------------------
# export data to CSVs
for df in dfs:
    dfs[df].to_csv('outputs/' + df + '.csv', index=False)

addressdf = pd.DataFrame({'name': addresses.keys(),'address': addresses.values()})
addressdf.to_csv('outputs/addresses.csv', index=False)
# ----------------------------------------------------------------------------------------

# Send Data to Database

In [ ]:
import psycopg2.extras as extras

# calculate how many rows of new data are to be inserted
print('New records:')
for df in dfs:
    data_check = 0
    data_check += len(dfs[df].index)
    print('-- ' + df + ' - ' + str(len(dfs[df].index)))


if data_check != 0:
    print('Uploading data to database...')
    # send data
    conn = psycopg2.connect(DB_CREDS)
    cursor = conn.cursor()
    for txtype in txtypes:
        if txtype == 'balance':
            tuples = [tuple(x) for x in dfs[txtype].to_numpy()]
            cols = '\",\"'.join(list(dfs[txtype].columns))
            # SQL query to execute
            query1 = 'truncate financials.' + txtype
            query2  = "INSERT INTO %s(\"%s\") VALUES %%s" % ('financials.'+ txtype, cols)
            cursor = conn.cursor()
            try:
                cursor.execute(query1)
                extras.execute_values(cursor, query2, tuples)
                conn.commit()
                print("Data succesfully uploaded.")
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()

        else:
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in dfs[txtype].to_numpy()]
            # Comma-separated dataframe columns
            cols = '\",\"'.join(list(dfs[txtype].columns))
            # SQL query to execute
            query  = "INSERT INTO %s(\"%s\") VALUES %%s" % ('financials.'+ txtype, cols)
            cursor = conn.cursor()
            try:
                extras.execute_values(cursor, query, tuples)
                conn.commit()
                print("Data succesfully uploaded.")
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()

    cursor.close()
    conn.close()
